In [21]:
# List all device
from tensorflow.python.client import device_lib
# print(device_lib.list_local_devices())

In [2]:
# Check available GPU
from keras import backend as K
K.tensorflow_backend._get_available_gpus()

Using TensorFlow backend.


['/job:localhost/replica:0/task:0/device:GPU:0']

In [3]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID";
# The GPU id to use, usually either "0" or "1";
os.environ["CUDA_VISIBLE_DEVICES"]="0";  

In [4]:
# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, GRU, Bidirectional
from keras.optimizers import SGD
import math
from sklearn.metrics import mean_squared_error
import pickle

In [5]:
with open("../data/dowjones_calculated/periods.txt", "rb") as fp:   # Unpickling
    dataset = pickle.load(fp)

In [6]:
X_train = dataset[0][0][0].values
y_train = dataset[0][0][1].values
X_test = dataset[1][0][0].values
y_test = dataset[1][0][1].values

In [7]:
y_train.shape

(750, 31)

In [8]:
X_test.shape

(250, 31)

In [9]:
y_test.shape

(250, 31)

In [10]:
# Reshaping X_train for efficient modelling
X_train = np.reshape(X_train, (X_train.shape[0],X_train.shape[1],1))
X_test = np.reshape(X_test, (X_test.shape[0],X_test.shape[1],1))

In [11]:
# The LSTM architecture
regressor = Sequential()
# First LSTM layer with Dropout regularisation
regressor.add(LSTM(units=25, return_sequences=True, input_shape=(X_train.shape[1],1)))
regressor.add(Dropout(0.1))
# Second LSTM layer
regressor.add(LSTM(units=50))
regressor.add(Dropout(0.2))
# The output layer
regressor.add(Dense(31, activation='softmax'))
# Compiling the RNN
regressor.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

# Fitting to the training set
regressor.fit(X_train,y_train,epochs=5,batch_size=32)

Epoch 1/5
750/750 [==============================] - 2s 3ms/step - loss: 1.7877 - acc: 0.4839
Epoch 2/5
750/750 [==============================] - 1s 1ms/step - loss: 1.7863 - acc: 0.4839
Epoch 3/5
750/750 [==============================] - 1s 1ms/step - loss: 1.7847 - acc: 0.4839
Epoch 4/5
750/750 [==============================] - 1s 1ms/step - loss: 1.7834 - acc: 0.4839
Epoch 5/5
750/750 [==============================] - 1s 1ms/step - loss: 1.7813 - acc: 0.4839


In [12]:
# Evaluating our model
predicted = regressor.predict(X_test)

In [13]:
y_test.shape

(250, 31)

In [14]:
predicted.shape

(250, 31)

In [15]:
y_test[0]

array([0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0,
       1, 1, 1, 1, 1, 0, 1, 0, 1])

In [16]:
predicted[0]

array([0.02727226, 0.03024466, 0.02902656, 0.03052546, 0.03134515,
       0.03269014, 0.03261134, 0.03468414, 0.0320346 , 0.03242021,
       0.02859943, 0.03336369, 0.03127077, 0.03247953, 0.03175566,
       0.02965897, 0.0315391 , 0.03060119, 0.03232913, 0.03355063,
       0.03380479, 0.03139265, 0.0323641 , 0.0337854 , 0.03469658,
       0.03353666, 0.03023486, 0.03495984, 0.03796116, 0.03496579,
       0.03429551], dtype=float32)

In [17]:
predicted[:3]

array([[0.02727226, 0.03024466, 0.02902656, 0.03052546, 0.03134515,
        0.03269014, 0.03261134, 0.03468414, 0.0320346 , 0.03242021,
        0.02859943, 0.03336369, 0.03127077, 0.03247953, 0.03175566,
        0.02965897, 0.0315391 , 0.03060119, 0.03232913, 0.03355063,
        0.03380479, 0.03139265, 0.0323641 , 0.0337854 , 0.03469658,
        0.03353666, 0.03023486, 0.03495984, 0.03796116, 0.03496579,
        0.03429551],
       [0.03253011, 0.02454136, 0.02764754, 0.02910126, 0.03274618,
        0.03476344, 0.02591562, 0.0380439 , 0.03285912, 0.0368184 ,
        0.02784164, 0.0289867 , 0.02587471, 0.02718208, 0.02535576,
        0.03744223, 0.03369911, 0.03083045, 0.02635411, 0.03128981,
        0.03681538, 0.0293224 , 0.03661219, 0.03300788, 0.04182931,
        0.03476038, 0.04325507, 0.03255562, 0.03356599, 0.03080646,
        0.03764576],
       [0.01680532, 0.02422477, 0.01774263, 0.02654435, 0.02671684,
        0.02447953, 0.02459644, 0.02501129, 0.02928063, 0.02929122,
      

In [18]:
label = predicted > 0.5
label = label * 1 # Convert boolean to int
label

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [19]:
(sum(y_test == label)/len(y_test)).mean()

0.4838709677419356

In [20]:
# from keras.models import load_model

# regressor.save('../model/LSTM/my_model.h5')  # creates a HDF5 file 'my_model.h5'
# del regressor  # deletes the existing model

# # returns a compiled model
# # identical to the previous one
# regressor1 = load_model('../model/LSTM/my_model.h5')
# regressor1.predict(X_test)